## データのロード

In [1]:
import numpy as np

from tensorflow import keras
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

#from keras.datasets import mnist
#(X_train, y_train), (X_test, y_test) = mnist.load_data()

/Users/teruitakahiro/opt/anaconda3/envs/data_science/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/teruitakahiro/opt/anaconda3/envs/data_science/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/teruitakahiro/opt/anaconda3/envs/data_science/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int1

In [2]:
# 平滑化
    
X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)

In [3]:
# 前処理

#X_train = X_train.astype(np.float)
#X_test = X_test.astype(np.float)
#X_train /= 255
#X_test /= 255

In [4]:
# 8:2にtrain_test_split

from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

# 【問題1】チャンネル数を1に限定した1次元畳み込み層クラスの作成

In [5]:
class SimpleConv1d():
    
    """
    ノード数n_nodes1からn_nodes2への全結合層
    Parameters
    ----------
    n_nodes1 : int
      前の層のノード数
    n_nodes2 : int
      後の層のノード数
    initializer : 初期化方法のインスタンス
    optimizer : 最適化手法のインスタンス
    """
    
    def __init__(self, lr=0.01):
        
        #self.optimizer = optimizer
        
        # 仮
        self.w = np.array([3, 5, 7])
        self.b = np.array([1])
        
        self.X = None
        
        self.dW = None
        self.dB = None
        
        self.pad = None
        self.stride = None
        self.FW = None

        self.lr = lr
        
        self.N_out = None
    
    
    def conv_forward(self, X):
        """
        フォワード
        
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, n_nodes1)
            入力
            
        Returns
        ----------
        A : 次の形のndarray, shape (batch_size, n_nodes2)
            出力
        """
    
        # 仮
        self.pad = 1
        self.stride = 1
        
        # フィルタの幅は重みのサイズ（今回は幅３）
        self.FW = len(self.w)
        
        # 入力Xをコントラクタに保存しておく
        self.X = X
              
        # conv後の出力のサイズ（self.N_out）を取っておく
        self.N_out = (len(X) + 2 * self.pad - self.FW) //  self.stride + 1
        
        # 入力にパディング
        X = np.pad(X, self.pad)

        # 出力の空箱を用意する
        out = np.zeros(self.N_out)
        
        # 入力をフィルタサイズに切り出し、フィルタの要素毎に積を取り、それを出力のi番目の要素としたい
        for i in range(self.N_out):
            # 入力のi番目からフィルタサイズ分、ストライド幅毎に、Xとwの積をとる
            out[i] += np.dot(X[i : i+self.FW], self.w.T)
            # ！！！ストライド未実装！！！
            
            print(out)
            
        # バイアスを足す    
        out += self.b
        

        return out
    
    
    def conv_backward(self, dA):
        
        """
        バックワード
        
        Parameters
        ----------
        dA : 次の形のndarray, shape (batch_size, n_nodes2)
            後ろから流れてきた勾配
            
        Returns
        ----------
        dZ : 次の形のndarray, shape (batch_size, n_nodes1)
            前に流す勾配
            
        """
        
        # XについてのLの微分
        delta_x = np.zeros(self.N_out)
        for i in range(len(dA)):
            delta_x[i : i + self.FW] += dA[i] * self.w
        print("delta_x" + str(delta_x))
            
        # BについてのLの微分
        delta_b = 0
        for i in range(len(dA)):
            delta_b += np.sum(dA[i], axis=0)
        print("delta_b" + str(delta_b))    
            
        # WについてのLの微分
        delta_w = np.zeros(len(self.w))
        for i in range(len(dA)):
            delta_w += dA[i] * self.X[i : i + self.FW]
        print("delta_w" + str(delta_w))
        
        # optimizerに流す前にデータ型を揃える
        self.w = self.w.astype(np.float64)
        self.b = self.b.astype(np.float64)
        
        # 更新
        self.w -= self.lr * delta_w
        self.b -= self.lr * delta_b
        #self.optimizer.update(self)
        
        print("self.w" + str(self.w))
        print("self.b" + str(self.b))
        
        return delta_x

## 【問題2】1次元畳み込み後の出力サイズの計算

In [6]:
pad = 1
stride  = 3
FW = 3

def calc_kernel_size(X):
    
    N_out = (X.shape[1] + 2 * pad - FW) //  stride + 1
    return N_out

In [7]:
calc_kernel_size(X_train)

262

## 【問題3】小さな配列での1次元畳み込み層の実験

In [8]:
s_conv1d = SimpleConv1d()

In [9]:
# forward

x = np.array([1, 2, 3, 4])

s_conv1d.conv_forward(x)

[19.  0.  0.  0.]
[19. 34.  0.  0.]
[19. 34. 49.  0.]
[19. 34. 49. 29.]


array([20., 35., 50., 30.])

In [10]:
# backward

delta_a = np.array([10, 20])

s_conv1d.conv_backward(delta_a)

delta_x[ 30. 110. 170. 140.]
delta_b30
delta_w[ 50.  80. 110.]
self.w[2.5 4.2 5.9]
self.b[0.7]


array([ 30., 110., 170., 140.])

In [11]:
# トイデータ
x = np.array([1, 2, 3, 4])
w = np.array([3, 5, 7])
a = np.empty((2, 3))

In [12]:
ind0 = np.array([0, 1, 2]).astype(np.int)
ind1 = np.array([1, 2, 3]).astype(np.int)

print(ind0)
print(ind1)

[0 1 2]
[1 2 3]


/Users/teruitakahiro/opt/anaconda3/envs/data_science/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """Entry point for launching an IPython kernel.
/Users/teruitakahiro/opt/anaconda3/envs/data_science/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32

In [13]:
FW=3 # フィルタ幅

x[0:FW]

array([1, 2, 3])

In [14]:
a[0] = x[ind0]*w # x[indexes0]は([1, 2, 3])である
a[1] = x[ind1]*w # x[indexes1]は([2, 3, 4])である

a = a.sum(axis=1)
a

array([34., 49.])

In [15]:
x = np.array([1, 2, 3, 4])
indexes = np.array([[0, 1, 2], [1, 2, 3]]).astype(np.int)
print(x[indexes]) # ([[1, 2, 3], [2, 3, 4]])

[[1 2 3]
 [2 3 4]]


/Users/teruitakahiro/opt/anaconda3/envs/data_science/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


 ## 【問題4】チャンネル数を限定しない1次元畳み込み層クラスの作成

### forward

In [16]:
x = np.array([[1, 2, 3, 4], [2, 3, 4, 5]]) # shape(2, 4)で、（入力チャンネル数、特徴量数）である。
w = np.ones((3, 2, 3)) # 例の簡略化のため全て1とする。(出力チャンネル数、入力チャンネル数、フィルタサイズ)である。
b = np.array([1, 2, 3]) # （出力チャンネル数）

In [17]:
FW = 3 #フィルタ幅
C = 3 #出力チャネル数
N_out = 2 #出力サイズ
out = np.zeros((C, N_out)) #出力

In [18]:
# ① 入力xの（N_in=0, F=0~3）要素とwの（C=0, N_in=0, FW=:）要素の積を取り、出力outの（C=0, N_out=0）要素に加える
out[0, 0] += (x[0, 0 : 0+FW] * w[0, 0, :]).sum()
out

array([[6., 0.],
       [0., 0.],
       [0., 0.]])

In [19]:
# ② 入力xの（N_in=0, F=1~4）要素とwの（C=0, N_in=0, FW=:）要素の積を取り、出力outの（C=0, N_out=1）要素に加える
# F = N_out : N_out + FW
out[0, 1] += (x[0, 1 : 1+FW] * w[0, 0, :]).sum()
out

array([[6., 9.],
       [0., 0.],
       [0., 0.]])

In [20]:
# ③ 入力xの（N_in=1, F=0~3）要素とwの（C=0, N_in=1, FW=:）要素の積を取り、出力outの（C=0, N_out=0）要素に加える
out[0, 0] += (x[1, 0 : 0+FW] * w[0, 1, :]).sum()
out

array([[15.,  9.],
       [ 0.,  0.],
       [ 0.,  0.]])

In [21]:
# ④ 入力xの（N_in=1, F=1~4）要素とwの（C=0, N_in=1, FW=:）要素の積を取り、出力outの（C=0, N_out=1）要素に加える
out[0, 1] += (x[1, 1 : 1+FW] * w[0, 1, :]).sum()
out

array([[15., 21.],
       [ 0.,  0.],
       [ 0.,  0.]])

In [22]:
# ⑤ 出力outの（C=0, N_out=:）要素にバイアスを加える
out[0, :] += b[0]
out

array([[16., 22.],
       [ 0.,  0.],
       [ 0.,  0.]])

In [23]:
# ⑥ ↑の処理をfor ループ化

FW = 3 #フィルタ幅
C = 3 #出力チャネル数
N_out = 2 #出力サイズ
N_in = x.shape[0]
out = np.zeros((C, N_out)) #出力

for i in range(N_in):
    for o in range(N_out):
        out[0, o] += (x[i, o : o+FW] * w[0, i, :]).sum()
out[0, :] += b[0]
print(out)

[[16. 22.]
 [ 0.  0.]
 [ 0.  0.]]


In [24]:
# ⑦ ↑を入力チャネル数分（C回）行う

FW = 3 #フィルタ幅
C = 3 #出力チャネル数
N_out = 2 #出力サイズ
out = np.zeros((C, N_out)) #出力

for c in range(C):
    for i in range(N_in):
        for o in range(N_out):
            out[c, o] += (x[i, o : o+FW] * w[c, i, :]).sum()
    out[c, :] += b[c]
print(out)

[[16. 22.]
 [17. 23.]
 [18. 24.]]


In [25]:
a = np.array([[16, 22], [17, 23], [18, 24]]) # shape(3, 2)で、（出力チャンネル数、入力チャネル数）である。

## 【問題5】（アドバンス課題）パディングの実装

### padding

In [26]:
x = np.array([[1, 2, 3, 4], [2, 3, 4, 5]])

In [27]:
np.pad(x, pad)

array([[0, 0, 0, 0, 0, 0],
       [0, 1, 2, 3, 4, 0],
       [0, 2, 3, 4, 5, 0],
       [0, 0, 0, 0, 0, 0]])

In [28]:
# 入力チャネル数方向にゼロパディング
np.pad(x, [(pad, pad), (0, 0)])

array([[0, 0, 0, 0],
       [1, 2, 3, 4],
       [2, 3, 4, 5],
       [0, 0, 0, 0]])

In [29]:
# 特徴量方向にゼロパディング（今回はこれを使いたい）
np.pad(x, [(0, 0), (pad, pad)])

array([[0, 1, 2, 3, 4, 0],
       [0, 2, 3, 4, 5, 0]])

In [30]:
x_padded = np.pad(x, [(0, 0), (pad, pad)])
x_padded

array([[0, 1, 2, 3, 4, 0],
       [0, 2, 3, 4, 5, 0]])

In [31]:
FW = 3 #フィルタ幅
C = 3 #出力チャネル数
stride = 1
pad = 1

# パディング後の出力サイズの計算（strideは１で固定）＊パディング前のxのサイズを使うことに注意
N_out = (x.shape[1] + 2 * pad - FW) //  stride + 1
out = np.zeros((C, N_out)) #出力

N_out, out

(4,
 array([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]))

In [32]:
x_padded = np.pad(x, [(0, 0), (pad, pad)])

for c in range(C):
    for i in range(N_in):
        for o in range(N_out):
            out[c, o] += (x_padded[i, o : o+FW] * w[c, i, :]).sum() # padされたxに置き換えている
    out[c, :] += b[c]
print(out)

[[ 9. 16. 22. 17.]
 [10. 17. 23. 18.]
 [11. 18. 24. 19.]]


## 【問題7】（アドバンス課題）任意のストライド数

### stride

In [33]:
x_padded

array([[0, 1, 2, 3, 4, 0],
       [0, 2, 3, 4, 5, 0]])

In [34]:
x_padded = np.pad(x, [(0, 0), (pad, pad)])

FW = 3 #フィルタ幅
C = 3 #出力チャネル数
stride = 3
pad = 1

N_out = (x.shape[1] + 2 * pad - FW) //  stride + 1
out = np.zeros((C, N_out)) #出力

for c in range(C):
    for i in range(N_in):
        for o in range(N_out):
            out[c, o] += (x_padded[i, (o*stride) : (o*stride)+FW] * w[c, i, :]).sum() # stride=3 の間隔で処理を行うように改変
            print(out)
            print("-"*12)
    out[c, :] += b[c]
print(out)

[[3. 0.]
 [0. 0.]
 [0. 0.]]
------------
[[3. 7.]
 [0. 0.]
 [0. 0.]]
------------
[[8. 7.]
 [0. 0.]
 [0. 0.]]
------------
[[ 8. 16.]
 [ 0.  0.]
 [ 0.  0.]]
------------
[[ 9. 17.]
 [ 3.  0.]
 [ 0.  0.]]
------------
[[ 9. 17.]
 [ 3.  7.]
 [ 0.  0.]]
------------
[[ 9. 17.]
 [ 8.  7.]
 [ 0.  0.]]
------------
[[ 9. 17.]
 [ 8. 16.]
 [ 0.  0.]]
------------
[[ 9. 17.]
 [10. 18.]
 [ 3.  0.]]
------------
[[ 9. 17.]
 [10. 18.]
 [ 3.  7.]]
------------
[[ 9. 17.]
 [10. 18.]
 [ 8.  7.]]
------------
[[ 9. 17.]
 [10. 18.]
 [ 8. 16.]]
------------
[[ 9. 17.]
 [10. 18.]
 [11. 19.]]


## 【問題6】（アドバンス課題）ミニバッチへの対応

#### mini_batch

In [35]:
# バッジサイズ N=2 の入力を生成

x_3d = np.random.randint(1, 9, (2, 2, 4))
x_3d.shape, x_3d # shape  =（N, N_in, F） 

((2, 2, 4),
 array([[[6, 6, 6, 1],
         [2, 7, 7, 4]],
 
        [[5, 2, 8, 4],
         [4, 2, 1, 3]]]))

In [36]:
# Feature方向にパディング

x_3d_padded = np.pad(x_3d, [(0, 0), (0, 0), (pad, pad)])
x_3d_padded.shape, x_3d_padded

((2, 2, 6),
 array([[[0, 6, 6, 6, 1, 0],
         [0, 2, 7, 7, 4, 0]],
 
        [[0, 5, 2, 8, 4, 0],
         [0, 4, 2, 1, 3, 0]]]))

In [37]:
b

array([1, 2, 3])

In [38]:
N = 2 #ミニバッチ数
FW = 3 #フィルタ幅
C = 3 #出力チャネル数

stride = 3
pad = 1

N_out = (x_3d.shape[2] + 2 * pad - FW) //  stride + 1
out = np.zeros((N, C, N_out)) #出力

for n in range(N):
    for c in range(C):
        for i in range(N_in):
            for o in range(N_out):
                out[n, c, o] += (x_3d_padded[n, i, (o*stride) : (o*stride)+FW] * w[c, i, :]).sum()
        out[n, c, :] += b[c]
print(out)

[[[22. 19.]
  [23. 20.]
  [24. 21.]]

 [[14. 17.]
  [15. 18.]
  [16. 19.]]]


### backward

In [39]:
x = np.array([[1, 2, 3, 4], [2, 3, 4, 5]]) # shape(2, 4)で、（入力チャンネル数、特徴量数）である。
w = np.ones((3, 2, 3)) # 例の簡略化のため全て1とする。(出力チャンネル数、入力チャンネル数、フィルタサイズ)である。
b = np.array([1, 2, 3]) # （出力チャンネル数）

In [40]:
dA = np.array([[9, 11], [32, 35], [52, 56]])
dA.shape #（C, N_out）

(3, 2)

In [41]:
dA[0]

array([ 9, 11])

In [42]:
dA[0, 0]

9

In [43]:
dA[0, 1]

11

#### delta_x

In [44]:
delta_x = np.zeros((x.shape))
delta_x

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [45]:
# ① delta_x の N_in=0 の F=0~2 に、C=0, N_out=0のdAと C=0, N=0 のwの各要素を掛け合わたものを足し込む。
delta_x[0, 0 : 0 + FW] += dA[0, 0]*w[0, 0, :]
delta_x

array([[9., 9., 9., 0.],
       [0., 0., 0., 0.]])

In [46]:
# ② delta_x の N_in=0 の F=1~3 に、C=0, N_out=1のdAと C=0, N=0 のwの各要素を掛け合わたものを足し込む。
delta_x[0, 1 : 1 + FW] += dA[0, 1]*w[0, 0, :]
delta_x

array([[ 9., 20., 20., 11.],
       [ 0.,  0.,  0.,  0.]])

In [47]:
# ③ delta_x の N_in=1 の F=0~2 に、C=0, N_out=0のdAと C=0, N=1 のwの各要素を掛け合わたものを足し込む。
delta_x[1, 0 : 0 + FW] += dA[0, 0]*w[0, 1, :]
delta_x

array([[ 9., 20., 20., 11.],
       [ 9.,  9.,  9.,  0.]])

In [48]:
# ④ delta_x の N_in=1 の F=1~3 に、C=0, N_out=1のdAと C=0, N=1 のwの各要素を掛け合わたものを足し込む。
delta_x[1, 1 : 1 + FW] += dA[0, 1]*w[0, 1, :]
delta_x

array([[ 9., 20., 20., 11.],
       [ 9., 20., 20., 11.]])

In [49]:
# ⑤↑をfor ループ化

N_out= dA.shape[1] # 出力サイズ:2
FW = w.shape[2] # フィルタ幅:3
F = x.shape[1] #特徴量:4
N_in = x.shape[0] #入力チャネル数:2
delta_x = np.zeros((N_in, F))

for i in range(N_in):
    for o in range(N_out):
        delta_x[i, o : o+FW] += dA[0, o]*w[0, i, :]
print(delta_x)

[[ 9. 20. 20. 11.]
 [ 9. 20. 20. 11.]]


In [50]:
# ↑をC回ループ処理を行う
C = dA.shape[0] # 入力チャネル数:3
N_out= dA.shape[1] # 出力サイズ:2
FW = w.shape[2] # フィルタ幅:3
F = x.shape[1] #特徴量:4
N_in = x.shape[0] #入力チャネル数:2
delta_x = np.zeros((N_in, F))

for c in range(C):
    for i in range(N_in):
        for o in range(N_out):
            delta_x[i, o : o + FW] += dA[c, o]*w[c, i, :]
print(delta_x)

[[ 93. 195. 195. 102.]
 [ 93. 195. 195. 102.]]


#### delta_w

In [51]:
x = np.array([[1, 2, 3, 4], [2, 3, 4, 5]])
x

array([[1, 2, 3, 4],
       [2, 3, 4, 5]])

In [52]:
FW = 3

delta_w = np.zeros((w.shape))
delta_w.shape, delta_w # （C, N, W）

((3, 2, 3),
 array([[[0., 0., 0.],
         [0., 0., 0.]],
 
        [[0., 0., 0.],
         [0., 0., 0.]],
 
        [[0., 0., 0.],
         [0., 0., 0.]]]))

In [53]:
# ① dAの（C=0, N_out=0）要素とxの（N_in=0、F=0 : 3）要素を掛け、delta_wの（C=0, N_in=0, FW=:）要素に足す
delta_w[0, 0, :] += dA[0, 0] * x[0, 0 : 0+FW]
delta_w

array([[[ 9., 18., 27.],
        [ 0.,  0.,  0.]],

       [[ 0.,  0.,  0.],
        [ 0.,  0.,  0.]],

       [[ 0.,  0.,  0.],
        [ 0.,  0.,  0.]]])

In [54]:
# ② dAの（C=0, N_out=1）要素とxの（N_in=0、F=1 : 4）要素を掛け、delta_wの（C=0, N_in=0, FW=:）要素に足す
delta_w[0, 0, :] += dA[0, 1] * x[0, 1 : 1 + FW]
delta_w

array([[[31., 51., 71.],
        [ 0.,  0.,  0.]],

       [[ 0.,  0.,  0.],
        [ 0.,  0.,  0.]],

       [[ 0.,  0.,  0.],
        [ 0.,  0.,  0.]]])

In [55]:
# ③ dAの（C=0, N_out=0）要素とxの（N_in=1、F=0 : 3）要素を掛け、delta_wの（C=0, N_in=1, FW=:）要素に足す
delta_w[0, 1, :] += dA[0, 0] * x[1, 0 : 0+FW]
delta_w

array([[[31., 51., 71.],
        [18., 27., 36.]],

       [[ 0.,  0.,  0.],
        [ 0.,  0.,  0.]],

       [[ 0.,  0.,  0.],
        [ 0.,  0.,  0.]]])

In [56]:
# ④ dAの（C=0, N_out=1）要素とxの（N_in=1、F=1 : 4）要素を掛け、delta_wの（C=0, N_in=1, FW=:）要素に足す
delta_w[0, 1, :] += dA[0, 1] * x[1, 1 : 1+FW]
delta_w

array([[[31., 51., 71.],
        [51., 71., 91.]],

       [[ 0.,  0.,  0.],
        [ 0.,  0.,  0.]],

       [[ 0.,  0.,  0.],
        [ 0.,  0.,  0.]]])

In [57]:
# ⑤↑をforループ化
N_in = 2 #入力
N_out= 2 # 出力
FW = 3 # フィルタ幅
delta_w = np.zeros((w.shape))

for i in range(N_in):
    for o in range(N_out):
        delta_w[0, i, :] += dA[0, o] * x[i, o : o+FW]
print(delta_w)

[[[31. 51. 71.]
  [51. 71. 91.]]

 [[ 0.  0.  0.]
  [ 0.  0.  0.]]

 [[ 0.  0.  0.]
  [ 0.  0.  0.]]]


In [58]:
# ⑥同様の処理をチャネル数分（C回）行う
C = 3 #チャネル
N_in = 2 #入力
N_out= 2 # 出力
FW = 3 # フィルタ幅
delta_w = np.zeros((w.shape))

for c in range(C):
    for i in range(N_in):
        for o in range(N_out):
            delta_w[c, i, :] += dA[c, o] * x[i, o : o+FW]
print(delta_w)

[[[ 31.  51.  71.]
  [ 51.  71.  91.]]

 [[102. 169. 236.]
  [169. 236. 303.]]

 [[164. 272. 380.]
  [272. 380. 488.]]]


#### delta_b

In [59]:
print(b)
print(b.shape)

[1 2 3]
(3,)


In [60]:
delta_b = np.zeros((b.shape))
delta_b

array([0., 0., 0.])

In [61]:
# ① dAの（C=0）要素の総和を、delta_bの（C=0）要素とする
delta_b[0] += dA[0, :].sum()
delta_b

array([20.,  0.,  0.])

In [62]:
# 同様の処理ををチャネル数分（C回）行う
C = 3
delta_b = np.zeros((b.shape))

for c in range(C):
    delta_b[c] += dA[c, :].sum()
print(delta_b)

[ 20.  67. 108.]


In [63]:
class Conv1d():
    
    """
    ノード数n_nodes1からn_nodes2への全結合層
    Parameters
    ----------
    n_nodes1 : int
      前の層のノード数
    n_nodes2 : int
      後の層のノード数
    initializer : 初期化方法のインスタンス
    optimizer : 最適化手法のインスタンス
    """
    
    def __init__(self, stride, pad, lr=0.01):
        
        #self.optimizer = optimizer
        
        #forward
        self.X = None
        
        self.w = np.ones((3, 2, 3)) # 今回はコンストラクタ中に組み込んでいる（実際はInitializerを使う）
        self.b = np.array([1, 2, 3]) 
        
        self.N_in = None #入力チャネル
        self.C = None # 出力チャネル
        self.FW = None # フィルタサイズ（幅のみ）
        self.F = None
        self.N_out = None
        
        self.pad = pad
        self.stride = stride

        self.dW = None
        self.dB = None
        
        self.lr = lr
    
    
    def calc_kernel_size(self, X):
        
        """
        出力のサイズを計算
        """
        
        N_out = (self.F - self.FW) //  self.stride + 1 # self.Fはpad後のXの幅
        return N_out

    
    def conv_forward(self, X):
        """
        フォワード
        
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, n_nodes1)
            入力
            
        Returns
        ----------
        A : 次の形のndarray, shape (batch_size, n_nodes2)
            出力
        """
        # ストライド・パディング
        stride = self.stride
        pad = self.pad
               
        # 入力にパディング（幅方向のみ）
        X = np.pad(X, [(0, 0), (pad, pad)]) 
        
        # 重みの型を取る
        C, N_in, FW = self.w.shape
        
        self.N_in = N_in
        self.C = C
        self.FW = FW
        
        # 入力の型を取る
        N_in, F = X.shape
        
        #self.N = N
        self.N_in = N_in
        self.F = F #pad後の特徴量
        
        # pad後の入力Xをコントラクタに保存
        self.X = X
        
        w = self.w
        b = self.b

        # 出力の型を用意
        N_out = self.calc_kernel_size(X)
        self.N_out = N_out
        out = np.zeros((C, N_out))
        
        for c in range(C):
            for i in range(N_in):
                for o in range(N_out):
                        
                    out[c, o] += (X[i, (o*stride) : (o*stride)+FW] * w[c, i, :]).sum()
            out[c, :] += b[c]
        
        return out
    
    
    def conv_backward(self, dA):
        
        """
        バックワード
        
        Parameters
        ----------
        dA : 次の形のndarray, shape (batch_size, n_nodes2)
            後ろから流れてきた勾配
            
        Returns
        ----------
        dZ : 次の形のndarray, shape (batch_size, n_nodes1)
            前に流す勾配
            
        """
        
        C = self.C
        N_out= self.N_out
        FW = self.FW
        F = self.F
        N_in = self.N_in
        
        # ストライド・パディング
        stride = self.stride
        pad = self.pad
        
        
        # delta_x
        delta_x = np.zeros((self.X.shape))
        
        for c in range(C):
            for i in range(N_in):
                for o in range(N_out):
                    delta_x[i, o : o + FW] += dA[c, o]*w[c, i, :]
         
        # paddingした分をリサイズし、元の入力サイズと同じにする
        delta_x = delta_x[:, pad : -pad]

        
        # delta_b
        delta_b = np.zeros((self.b.shape))

        for c in range(C):
            delta_b[c] += dA[c, :].sum()
        
            
        # delta_w
        delta_w = np.zeros((self.w.shape))
        
        for c in range(C):
            for i in range(N_in):
                for o in range(N_out):
                    delta_w[c, i, :] += dA[c, o] * self.X[i, o : o+FW]
        
        
        # optimizerに流す前にデータ型を揃える
        self.w = self.w.astype(np.float64)
        self.b = self.b.astype(np.float64)
        
        # 更新
        self.w -= self.lr * delta_w
        self.b -= self.lr * delta_b
        #self.optimizer.update(self)
        
        print("self.w" + str(self.w))
        print("self.b" + str(self.b))
        
        return delta_x

In [64]:
conv1d = Conv1d(stride=1, pad=1)

In [65]:
# forward
x = np.array([[1, 2, 3, 4], [2, 3, 4, 5]])
conv1d.conv_forward(x).shape, conv1d.conv_forward(x)

((3, 4),
 array([[ 9., 16., 22., 17.],
        [10., 17., 23., 18.],
        [11., 18., 24., 19.]]))

In [66]:
# backward（1pad, 1stride）
#dA = np.array([[9, 11], [32, 35], [52, 56]])
dA = np.random.randint(1, 9, (3, 4))
dA.shape, dA

((3, 4),
 array([[7, 3, 7, 7],
        [6, 2, 3, 2],
        [4, 3, 5, 6]]))

In [67]:
# 入力と同じ形で返す（2, 4）

dZ = conv1d.conv_backward(dA)
dZ

self.w[[[0.62 0.38 0.49]
  [0.45 0.14 0.32]]

 [[0.86 0.73 0.7 ]
  [0.79 0.6  0.59]]

 [[0.69 0.51 0.63]
  [0.55 0.33 0.51]]]
self.b[0.76 1.87 2.82]


array([[25., 40., 38., 30.],
       [25., 40., 38., 30.]])

## 【問題8】学習と推定

In [372]:
#import numpy as np
#from keras.datasets import mnist

#(X_train, y_train), (X_test, y_test) = mnist.load_data()

from tensorflow import keras
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

In [373]:
# 平滑化
    
X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)

In [374]:
# 前処理

X_train = X_train.astype(np.float64)
X_test = X_test.astype(np.float64)
X_train /= 255
X_test /= 255

In [375]:
# 8:2にtrain_test_split

from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

# Layer

In [355]:
# 雛形

class FC:
    
    """
    ノード数n_nodes1からn_nodes2への全結合層
    Parameters
    ----------
    n_nodes1 : int
      前の層のノード数
    n_nodes2 : int
      後の層のノード数
    initializer : 初期化方法のインスタンス
    optimizer : 最適化手法のインスタンス
    """
    
    def __init__(self, n_nodes1, n_nodes2, initializer, optimizer):
        
        self.optimizer = optimizer
        
        self.W = initializer.W(n_nodes1, n_nodes2)
        self.B = initializer.B(n_nodes2)
       
        self.X = None
        self.dZ = None
        
    
    def forward(self, X):
        """
        フォワード
        
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, n_nodes1)
            入力
            
        Returns
        ----------
        A : 次の形のndarray, shape (batch_size, n_nodes2)
            出力
        """
        
        self.X = X
        
        # アフィン変換
        A = X.dot(self.W) + self.B

        return A
    
    
    def backward(self, dA):
        
        """
        バックワード
        
        Parameters
        ----------
        dA : 次の形のndarray, shape (batch_size, n_nodes2)
            後ろから流れてきた勾配
            
        Returns
        ----------
        dZ : 次の形のndarray, shape (batch_size, n_nodes1)
            前に流す勾配
            
        """
        # 前のノードに流す勾配
        self.dZ = np.dot(dA, self.W.T)
        
        # バイアスに流す勾配
        self.dB = np.sum(dA, axis=0)
        
        # 重み行列に流す勾配
        self.dW = np.dot(self.X.T, dA)
        
        # 更新
        self.optimizer.update(self)
        
        return self.dZ

In [531]:
class Conv1d():
    
    """
    ノード数n_nodes1からn_nodes2への全結合層
    Parameters
    ----------
    n_nodes1 : int
      前の層のノード数
    n_nodes2 : int
      後の層のノード数
    initializer : 初期化方法のインスタンス
    optimizer : 最適化手法のインスタンス
    """
    
    def __init__(self, initializer, optimizer, N=20, O=1, C=1,  FW=3, stride=1, pad=0, lr=0.01):
        
        self.optimizer = optimizer
        
        #forward
        self.X = None
            
        self.N = N #サンプル数
        
        self.O = O
        self.C = C
        
        self.FW = FW # フィルタサイズ（幅のみ）
        self.F = None #特徴量数
        self.f_out = None #出力の特徴量数
        
        self.initializer = initializer
        self.W = self.initializer.W(N, O, C, FW) # (バッチサイズ、出力チャネル数、入力チャネル数、フィルタサイズ)
        self.B = self.initializer.B(N, O) # (バッチサイズ、出力チャネル数)
        
        self.pad = pad
        self.stride = stride
         
        self.dW = None
        self.dB = None
        
        self.lr = lr
    
    
    def calc_kernel_size(self, X):
        
        """
        出力のサイズを計算
        """
        
        out = (self.F - self.FW) //  self.stride + 1 # self.Fはpad後のXの幅
        return out

    
    def conv_forward(self, X):
        """
        フォワード
        
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, n_nodes1)
            入力
            
        Returns
        ----------
        A : 次の形のndarray, shape (batch_size, n_nodes2)
            出力
        """
        # ストライド・パディング
        stride = self.stride
        pad = self.pad
        
        N = self.N
        O = self.O
        C = self.C
        
        # 入力にパディング（幅方向のみ）
        X = np.pad(X, [(0, 0), (0, 0), (pad, pad)]) 

        # 入力の型を取る
        _, _, F = X.shape
        self.F = F
        self.X = X # backwardで使う
        
        w = self.W
        b = self.B
        
        # 出力の型を用意
        f_out = self.calc_kernel_size(X)
        self.f_out = f_out
        out = np.zeros((N, O, f_out))
        
        for n in range(N):
            for o in range(O):
                for c in range(C):
                    for f in range(f_out):
                        out[n, o, f] += (X[n, c, (f*stride) : (f*stride)+FW] * w[n, o, c, :]).sum()
                        out += b[n, :]
        
        return out
    
    
    def conv_backward(self, dA):
        
        """
        バックワード
        
        Parameters
        ----------
        dA : 次の形のndarray, shape (batch_size, n_nodes2)
            後ろから流れてきた勾配
            
        Returns
        ----------
        dZ : 次の形のndarray, shape (batch_size, n_nodes1)
            前に流す勾配
            
        """
        
        X = self.X
        f_out = self.f_out
        FW = self.FW
        C  = self.C
        O = self.O
        F = self.F
        w = self.W
            
        stride = self.stride
        pad = self.pad
        
   
        # delta_x
        delta_x = np.zeros((self.X.shape))
        
        for n in range(N):
            for o in range(O):
                for c in range(C):
                    for f in range(f_out):
                        delta_x[n, c, f*stride : f*stride+FW] += dA[n, o, f]*w[n, o, c, :]
                
        delta_x = delta_x[:, :, pad : -pad]
        
                
        # delta_b
        delta_b = np.zeros((self.B.shape))

        for n in range(N):
            for o in range(O):
                for f in range(f_out):
                        delta_b[n, o] += dA[n, o, :].sum()   
                        
        self.dB = delta_b        
        
           
        # delta_w
        delta_w = np.zeros((self.W.shape))
         
        for n in range(N):
            for o in range(O):
                for c in range(C):
                    for f in range(f_out):
                        delta_w[n, o, c,  :] += dA[n, o, f] * X[n, c, (f*stride) : (f*stride)+FW]
                                    
        self.dW = delta_w
                
        # optimizerに流す前にデータ型を揃える
        #self.W = self.W.astype(np.float64)
        #self.B = self.B.astype(np.float64)
        
        # 更新
        #self.W -= self.lr * delta_w
        #self.B -= self.lr * delta_b
        self.optimizer.update(self)
        
        return delta_x

# Activation

In [527]:
class Tanh:
         
    def forward(self, A):

        self.Z = (np.exp(A) - np.exp(-A)) / (np.exp(A) + np.exp(-A))
        return self.Z
    
    
    def backward(self, dZ):
        
        dA = dZ*(1 - self.Z)**2
        return dA

In [584]:
class Softmax:
    
    def forward(self, A):
        
        A = A - np.max(A, axis=1, keepdims=True)
        Z = np.exp(A) / np.sum(np.exp(A), axis=1, keepdims=True)
        return Z
        
        
    def backward(self, dZ, Y):
        
        batch_size = dZ.shape[0]
        dA = (dZ - Y) / batch_size
        
        loss = (-1)*np.sum(Y * np.log(dZ + 1e-7))  / batch_size
        
        
        return dA, loss

In [633]:
class ReLU:
    
    
    def __init__(self):
        
        self.mask = None
    
    
    def forward(self, A):

        self.mask = (A <= 0)
        Z = np.maximum(0, A)
        return Z
        
    
    def backward(self, dA):
        
        dA[self.mask] = 0
        return dA

In [576]:
class ReLU:
    
    def forward(self, A):

        self.A = A
        Z = np.maximum(0, A)
        return Z
        
    
    def backward(self, dZ):
        
        dA = np.where(self.A>0, dZ, 0)
        return dA

# Initializer

In [577]:
class ConvInitializer():
    
    """
    ガウス分布によるシンプルな初期化
    Parameters
    ----------
    sigma : float
      ガウス分布の標準偏差
    """
    
    def __init__(self, sigma=0.01):
        
        self.sigma = sigma
        
        
    def W(self, N, O, C, F):
        
        """
        重みの初期化
        
        Parameters
        ----------
        n_nodes1 : int
          前の層のノード数
        n_nodes2 : int
          後の層のノード数
          
        Returns
        ----------
        W :float
            重みの初期値
            
        """
        
        W = self.sigma * np.random.randn(N, O, C, F)
        
        return W
    
    
    def B(self, N, O):
        
        """
        バイアスの初期化
        
        Parameters
        ----------
        n_nodes2 : int
          後の層のノード数
          
        Returns
        ----------
        B :float
            バイアスの初期値
            
        """
        
        B = self.sigma * np.random.randn(N, O)
        
        return B

In [578]:
class SimpleInitializer():
    
    """
    ガウス分布によるシンプルな初期化
    Parameters
    ----------
    sigma : float
      ガウス分布の標準偏差
    """
    
    def __init__(self, sigma=0.01):
        
        self.sigma = sigma
        
        
    def W(self, n_nodes1, n_nodes2):
        
        """
        重みの初期化
        
        Parameters
        ----------
        n_nodes1 : int
          前の層のノード数
        n_nodes2 : int
          後の層のノード数
          
        Returns
        ----------
        W :float
            重みの初期値
            
        """
        
        W = self.sigma * np.random.randn(n_nodes1, n_nodes2)
        
        return W
    
    
    def B(self, n_nodes2):
        
        """
        バイアスの初期化
        
        Parameters
        ----------
        n_nodes2 : int
          後の層のノード数
          
        Returns
        ----------
        B :float
            バイアスの初期値
            
        """
        
        B = self.sigma * np.random.randn(n_nodes2)
        
        return B

In [579]:
class HeInitializer:
    
    """
    Heの初期値
      
    """
    def __init__(self, sigma):
        
        self.sigma = sigma
        
    
    
    
    def W(self, n_nodes1, n_nodes2):
        
        """
        重みの初期化
        
        Parameters
        ----------
        n_nodes1 : int
          前の層のノード数
        n_nodes2 : int
          後の層のノード数
          
        Returns
        ----------
        W :float
            重みの初期値
            
        """
        self.n_nodes1 = n_nodes1
        sigma = np.sqrt(2 / n_nodes1)
        W = sigma * np.random.randn(n_nodes1, n_nodes2)
        
        return W
    
    
    def B(self, n_nodes2):
        
        """
        バイアスの初期化
        
        Parameters
        ----------
        n_nodes2 : int
          後の層のノード数
          
        Returns
        ----------
        B :float
            バイアスの初期値
            
        """
        
        sigma = np.sqrt(2 / self.n_nodes1)
        B = sigma * np.random.randn(n_nodes2)
        
        return B

# Optimizer

In [81]:
class SGD():
    
    """
    確率的勾配降下法
    
    Parameters
    ----------
    lr : 学習率
    
    """
    
    def __init__(self, lr):
        
        self.lr = lr
        
        
    def update(self, layer):
        
        """
        ある層の重みやバイアスの更新
        
        Parameters
        ----------
        layer : 更新前の層のインスタンス
        
        """

        layer.W -= self.lr * layer.dW
        layer.B -= self.lr * layer.dB

In [82]:
class AdaGrad():
    
    """
    確率的勾配降下法
    
    Parameters
    ----------
    lr : 学習率
    
    """
    
    def __init__(self, lr):
        
        self.lr = lr
        self.hW = 1.
        self.hB = 1.
        
    def update(self, layer):
        
        """
        ある層の重みやバイアスの更新
        
        Parameters
        ----------
        layer : 更新前の層のインスタンス
        
        """
        
        self.hW += np.mean(layer.dW, axis=0)**2
        self.hB += ((np.mean(layer.dB, axis=0))**2).sum(axis=0)
        
        layer.W -= (self.lr / np.sqrt(self.hW)) * layer.dW
        layer.B -= (self.lr / np.sqrt(self.hB)) * layer.dB

# Utility

In [83]:
class GetMiniBatch:
    
    """
    ミニバッチを取得するイテレータ
    
    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
      
    """
    
    def __init__(self, X, y, batch_size = 20, seed=0):
        
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self._X = X[shuffle_index]
        self._y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
        
        
    def __len__(self):
        return self._stop
    
    
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self._X[p0:p1], self._y[p0:p1]
    
    
    def __iter__(self):
        self._counter = 0
        return self
    
    
    def __next__(self):
        
        if self._counter >= self._stop:
            raise StopIteration()
            
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self._X[p0:p1], self._y[p0:p1]

# Trainer

In [642]:
class ScratchConvolutionalNeuralNetrowkClassifier():
    
    """
    任意の構成で学習と推定が行えるCNNクラス
    
    """
    
    def __init__(self, fc_initializer, conv_initializer, optimizer, activator1, activator2, lr = 0.01,  batch_size = 20, n_features = 88, n_output = 10, sigma=0.01, n_epochs=20, verbose = True):
        
        self.verbose = verbose
        
        self.n_epochs = n_epochs
        self.batch_size = batch_size
        self.n_features = n_features
        self.n_output = n_output
        
        self.Conv1 = None
        self.Conv2 = None
        self.FC = None
        
        self.fc_initializer = fc_initializer
        self.conv_initializer = conv_initializer
        self.optimizer = optimizer
        self.lr = lr
        
        self.activator1 = activator1
        self.activator2 = activator2   
        
        self.sigma = sigma
        
        self.loss = None
        self.val_loss = None
        
        self.loss_hist = []
        self.val_loss_hist = []
        
        
    # 訓練時に１エポック分の処理を行う関数
    def forward_and_backward(self, X, y):
        
        # one-hot変換
        Y = np.identity(10)[y]
        
        # 入力Xの入力チャンネル次元を作る
        X = X[:, np.newaxis, :]  #(N, C, F)
        #print("X"+str(X.shape))
        
        # サンプルコード2 : フォワード
        A1 = self.Conv1.conv_forward(X)
        #print("A1"+str(A1.shape))
        
        Z1 = self.activation1.forward(A1)
        #print("Z1"+str(Z1.shape))
        
        A2 = self.Conv2.conv_forward(Z1)
        #print("A2"+str(A2.shape))
        
        Z2 = self.activation2.forward(A2)
        #print("Z2"+str(Z2.shape))
        
        # FC層に渡せる形に変換
        Z2 = Z2[:, -1]
        #print("Z2"+str(Z2.shape))
        
        #F1 = self.Flat.flat_forward(Z2)
        #print("F1"+str(F1.shape))
        
        A3 = self.FC.forward(Z2)
        #print("A3"+str(A3.shape))
        
        Z3 = self.activation3.forward(A3)      
        #print("Z3"+str(Z3.shape))
    
        # サンプルコード3 : バックワード
        dA3, self.loss = self.activation3.backward(Z3, Y) # 交差エントロピー誤差とソフトマックスを合わせている
        #print("dA3"+str(dA3.shape))
            
        dZ2 = self.FC.backward(dA3)
        #print("dZ2"+str(dZ2.shape))
        
        # チャンネル軸を増やす
        dZ2 = dZ2[:, np.newaxis, :]
        #print("dZ2"+str(dZ2.shape))
        
        dA2 = self.activation2.backward(dZ2)
        #print("dA2"+str(dA2.shape))
        
        dZ1 = self.Conv2.conv_backward(dA2)
        #print("dZ1"+str(dZ1.shape))
        
        dA1 = self.activation1.backward(dZ1)
        #print("dA1"+str(dA1.shape))
        
        dZ0 = self.Conv1.conv_backward(dA1) # dZ0は使用しない
        #print(dZ0.shape)
        
    
    
    # このエポックで更新した重みを使って検証する関数
    def forward_with_loss(self, X, y):
        
        Y = np.identity(10)[y]
        
        X = X[:, np.newaxis, :]  #(N, C, F)
        #print("X"+str(X.shape))
        
        A1 = self.Conv1.conv_forward(X)
        #print("A1"+str(A1.shape))
        
        Z1 = self.activation1.forward(A1)
        #print("Z1"+str(Z1.shape))
        
        A2 = self.Conv2.conv_forward(Z1)
        #print("A2"+str(A2.shape))
        
        Z2 = self.activation2.forward(A2)
        #print("Z2"+str(Z2.shape))
        
        Z2 = Z2[:, -1]
        #print("Z2"+str(Z2.shape))
        
        A3 = self.FC.forward(Z2)
        #print("A3"+str(A3.shape))
        
        Z3 = self.activation3.forward(A3)      
        #print("Z3"+str(Z3.shape))
    
        dA3, self.val_loss = self.activation3.backward(Z3, Y) 
        #print("dA3"+str(dA3.shape))
        
    
    def fit(self, X, y, X_val=None, y_val=None):
        
        """
        self.sigma : ガウス分布の標準偏差
        self.lr : 学習率
        self.n_nodes1 : 1層目のノード数
        self.n_nodes2 : 2層目のノード数
        self.n_output : 出力層のノード数
        """
        
        # 最初に全ノードを生成
        self.Conv1 = Conv1d(self.conv_initializer(self.sigma), self.optimizer(self.lr), N=20, FW=3, stride=3, pad=1)
        
        self.activation1 = self.activator1
        
        self.Conv2 = Conv1d(self.conv_initializer(self.sigma), self.optimizer(self.lr), N=20, FW=3, stride=3, pad=1)
        
        self.activation2 = self.activator2
        
        self.FC = FC(self.n_features, self.n_output, self.fc_initializer(self.sigma), self.optimizer(self.lr))
        self.activation3 = Softmax()
        
        # 訓練データのミニバッチを生成
        get_mini_batch_train = GetMiniBatch(X, y, batch_size=self.batch_size)
        
        # 検証データのミニバッチを生成
        get_mini_batch_val = GetMiniBatch(X_val, y_val, batch_size=self.batch_size)
        
        for epoch in range(self.n_epochs):
            
            for mini_X_train, mini_y_train in get_mini_batch_train:
                
                # 順伝播・クロスエントロピー誤差・逆伝播
                self.forward_and_backward(mini_X_train, mini_y_train)
                
            self.loss_hist.append(self.loss)
            
            # 検証
            mini_X_val, mini_y_val = get_mini_batch_val[0]
            self.forward_with_loss(mini_X_val, mini_y_val)
            self.val_loss_hist.append(self.val_loss)
        
            
            if self.verbose:

                print('#'*25)
                print('### Epoch %i'%(epoch+1))
                print('#'*25)
                
                print("訓練データの損失 : {}".format(self.loss))
                print("検証データの損失 : {}".format(self.val_loss))
                
                print()
                
                
    
    def predict(self, X):
        
        
        preds = []
        
        for i in range(int(X.shape[0] // self.batch_size)):
        
       
            X_batch = X[i*self.batch_size : i*self.batch_size + self.batch_size]     
        
            X_batch = X_batch[:, np.newaxis, :]

            A1 = self.Conv1.conv_forward(X_batch)
            #print("A1"+str(A1.shape))

            Z1 = self.activation1.forward(A1)
            #print("Z1"+str(Z1.shape))

            A2 = self.Conv2.conv_forward(Z1)
            #print("A2"+str(A2.shape))

            Z2 = self.activation2.forward(A2)
            #print("Z2"+str(Z2.shape))

            Z2 = Z2[:, -1]
            #print("Z2"+str(Z2.shape))

            A3 = self.FC.forward(Z2)
            #print("A3"+str(A3.shape))

            Z3 = self.activation3.forward(A3)
            #print("Z3"+str(Z3.shape))

            pred = Z3.argmax(axis=1)

            preds.append(pred)
            
        out = np.array(preds)
        out = out.ravel()
        
        return out

In [643]:
cnn = ScratchConvolutionalNeuralNetrowkClassifier(fc_initializer=HeInitializer,
                                                  conv_initializer=ConvInitializer,
                                                  optimizer = AdaGrad,
                                                  lr=1e-4,
                                                  sigma=0.1, 
                                                  activator1=ReLU(),
                                                  activator2=ReLU(),
                                                  n_epochs=20,
                                                  verbose = True
                                                  )

In [644]:
%%time

cnn.fit(X_train, y_train, X_val, y_val)

/Users/teruitakahiro/opt/anaconda3/envs/data_science/lib/python3.7/site-packages/ipykernel_launcher.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


#########################
### Epoch 1
#########################
訓練データの損失 : 7.991261578490051
検証データの損失 : 6.476156223491832

#########################
### Epoch 2
#########################
訓練データの損失 : 7.587401455372442
検証データの損失 : 5.838422538469379

#########################
### Epoch 3
#########################
訓練データの損失 : 6.892581230477594
検証データの損失 : 5.377587860477381

#########################
### Epoch 4
#########################
訓練データの損失 : 6.461883842080425
検証データの損失 : 4.8956526945189465

#########################
### Epoch 5
#########################
訓練データの損失 : 5.530525066631119
検証データの損失 : 4.3678804999491465

#########################
### Epoch 6
#########################
訓練データの損失 : 4.975883789704128
検証データの損失 : 4.172090510210234

#########################
### Epoch 7
#########################
訓練データの損失 : 4.789682403733623
検証データの損失 : 4.089806427981912

#########################
### Epoch 8
#########################
訓練データの損失 : 4.62531051261322
検証データの損失 : 3.995539625580281

###############

In [652]:
y_pred = cnn.predict(X_test)
len(y_pred), y_pred

(10000, array([7, 7, 3, ..., 3, 7, 7]))

In [655]:
len(y_test), y_test

(10000, array([7, 2, 1, ..., 4, 5, 6], dtype=uint8))

In [656]:
# Accuracy

from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
accuracy

0.0998